* 定义item数据结构
* 从json中取出

In [ ]:
import time

from qwikidata.entity import WikidataItem
from qwikidata.json_dump import WikidataJsonDump
from qwikidata.utils import dump_entities_to_json
import json

# P_OCCUPATION = "P106"
# Q_POLITICIAN = "Q82955"

# def has_occupation_politician(item: WikidataItem, truthy: bool = True) -> bool:
#     """Return True if the Wikidata Item has occupation politician."""
#     if truthy:
#         claim_group = item.get_truthy_claim_group(P_OCCUPATION)
#     else:
#         claim_group = item.get_claim_group(P_OCCUPATION)

#     occupation_qids = [
#         claim.mainsnak.datavalue.value["id"]
#         for claim in claim_group
#         if claim.mainsnak.snaktype == "value"
#     ]
#     return Q_POLITICIAN in occupation_qids


# create an instance of WikidataJsonDump
wjd_dump_path = "wikidata\wikidata-20231215-all.json.bz2"
wjd = WikidataJsonDump(wjd_dump_path)

# create an iterable of WikidataItem representing politicians
politicians = []
t1 = time.time()
for ii, entity_dict in enumerate(wjd):
    print(entity_dict)
    if entity_dict["type"] == "item":
        entity = WikidataItem(entity_dict)
        # if has_occupation_politician(entity):
        politicians.append(entity)
    if ii % 1000 == 0 and ii > 0:
        t2 = time.time()
        dt = t2 - t1
        print(
            "found {} politicians among {} entities [entities/s: {:.2f}]".format(
                len(politicians), ii, ii / dt
            )
        )
    if ii == 1:
        break


# write the iterable of WikidataItem to disk as JSON
out_fname = "wikidata\wikidata-20231215-all.json"
dump_entities_to_json(politicians, out_fname)

In [ ]:
from qwikidata.json_dump import WikidataJsonDump
from qwikidata.entity import WikidataItem
from qwikidata.utils import dump_entities_to_json
from tqdm import tqdm
import os
import json


wjd_dump_path = "wikidata\wikidata-20231215-all.json.bz2"
wjd = WikidataJsonDump(wjd_dump_path)

output_file_folder = "wikidata/20231215-en-label-description"
if not os.path.exists(output_file_folder):
    os.makedirs(output_file_folder)
batch_size = 1000000
wikidata_list_en=[]
for ii, entity_dict in tqdm(enumerate(wjd)):
    if entity_dict["type"] == "item":
        entity = WikidataItem(entity_dict)
        wikidata_list_en.append({
            'id':entity.entity_id,
            'language': 'en',
            'label': entity.get_label(),
            'description': entity.get_description()})
    if ii % batch_size == 0 and ii > 0:
        json_file= os.path.join(output_file_folder, 'wikidata-20231215-en-label-description-'+str(ii)+'.json')
        with open(json_file, 'w') as outfile:
            json.dump(wikidata_list_en, outfile)
        wikidata_list_en=[]
        print("saved {} entities".format(ii))

# 将最后一批数据保存到json文件中
json_file= os.path.join(output_file_folder, 'wikidata-20231215-en-label-description-'+str(ii)+'.json')
with open(json_file, 'w') as outfile:
    json.dump(wikidata_list_en, outfile)

In [59]:
# 将数据插入es
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from qwikidata.json_dump import WikidataJsonDump
from qwikidata.entity import WikidataItem
from qwikidata.utils import dump_entities_to_json
import json
import os
import time
from tqdm import tqdm

es = Elasticsearch(hosts="http://localhost:9200")
index_name = "wikidata-filter"
es.indices.create(index=index_name,ignore=400)

wjd_dump_path = "wikidata\wikidata-20231215-all.json.bz2"
wjd = WikidataJsonDump(wjd_dump_path)

batch_size=10000
batch_num=0
item_list=[]

property_list = set(['Q1656682','Q2221906','Q34770','Q22097341','Q5','Q43229','Q2424752','Q838948'])

for ii, entity_dict in tqdm(enumerate(wjd)):
    if entity_dict["type"] == "item":
        entity = WikidataItem(entity_dict)
        if ii< 15000000:
            continue
        try:
            # 判断es中是否已经存在
            if es.exists(index=index_name,id=entity.entity_id):
                continue

            # 判断是否存在英文wiki标题
            en_wiki_title = entity.get_enwiki_title()
            if not bool(en_wiki_title):
                continue

            # 判断是否存在英文标签
            en_label = entity.get_label('en')
            if not bool(en_label):
                continue
            # 判断是否存在英文描述
            en_description = entity.get_description('en')
            if not bool(en_description):
                continue
            
            # 判断是否有P31
            instance_list = entity.get_claim_group('P31')
            if len(instance_list) <=0:
                continue
            
            # 判断P31 是否在 list中
            instance_value_list = [claim.mainsnak.datavalue.value["id"] for claim in entity.get_truthy_claim_group('P31')]
            if not bool(property_list & set(instance_value_list)):
                continue
        except Exception as e:
            # 捕获任意异常
            print(e)
            continue

        item={
            '_id': entity.entity_id,
            '_index': index_name,
            'entity_id':entity.entity_id,
            'label':en_label,
            'description':en_description,
            'enwiki_title':en_wiki_title,
            'instance_of': str(instance_value_list)
        }
        item_list.append(item)
        # 每bs条插入
        if len(item_list) > batch_size:
            helpers.bulk(client=es, actions=item_list)
            batch_num+=1
            item_list=[]
            print(f"batch num:{batch_num} has been inserted.The data efficiency rate is {batch_num * batch_size / ii}")

# 插入余下的数据
helpers.bulk(client=es, actions=item_list)


c:\Users\lqw\miniconda3\envs\simcse\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  
13478381it [12:15:06, 305.58it/s]


KeyboardInterrupt: 

In [61]:
# es检索测试
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200")
index_name = "wikidata-filter"

query_id = 'Q2687336'

if es.exists(index=index_name,id=query_id):
    print("1")
else:
    print("2")


2


In [58]:
# 拆分不同chunks
from qwikidata.json_dump import WikidataJsonDump
from qwikidata.entity import WikidataItem
from qwikidata.utils import dump_entities_to_json

wjd_dump_path = "wikidata\wikidata-20231215-all.json.bz2"
wjd = WikidataJsonDump(wjd_dump_path)

wjd.create_chunks(num_lines_per_chunk=100)

FileNotFoundError: [WinError 2] 系统找不到指定的文件。